In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import healpy as hp
import h5py
import matplotlib.pyplot as plt
import matplotlib
from sobol_seq import i4_sobol
from scipy.stats import skew, kurtosis
from tqdm import tqdm

from msfm.utils import files, parameters, input_output
from msfm.grid_pipeline import GridPipeline
from msfm.utils.cosmogrid import get_cosmo_params_info

n_side = 512
n_pix = hp.nside2npix(n_side)

23-09-07 23:54:07    scales.py INF   Setting up healpy to run on 256 CPUs 


# CosmoGrid params

In [3]:
cosmo_params_info = get_cosmo_params_info("../../data/CosmoGridV1_metainfo.h5")
print(cosmo_params_info.dtype)

all_Om = cosmo_params_info["Om"]
all_s8 = cosmo_params_info["s8"]
all_w0 = cosmo_params_info["w0"]
all_sobol = cosmo_params_info["sobol_index"]

[('As', '<f8'), ('bary_Mc', '<f8'), ('bary_nu', '<f8'), ('H0', '<f8'), ('O_cdm', '<f8'), ('O_nu', '<f8'), ('Ob', '<f8'), ('Ol', '<f8'), ('Om', '<f8'), ('m_nu', '<f8'), ('ns', '<f8'), ('s8', '<f8'), ('w0', '<f8'), ('wa', '<f8'), ('delta', 'S128'), ('sobol_index', '<i4'), ('benchmark_type', 'S128'), ('id_param', '<i4'), ('path_par', 'S128'), ('box_size_Mpc_over_h', '<f8'), ('n_particles', '<i8'), ('n_shells', '<i4'), ('n_steps', '<i4')]


# Extend params

In [4]:
conf = files.load_config()
sobol_priors = parameters.get_prior_intervals(
    conf["analysis"]["params"]["sobol"] + conf["analysis"]["params"]["ia"] + conf["analysis"]["params"]["bg"]
)

all_bg = []
all_n_bg = []
for i_sobol in all_sobol:
    sobol_point, _ = i4_sobol(sobol_priors.shape[0], i_sobol)
    sobol_params = sobol_point * np.squeeze(np.diff(sobol_priors)) + sobol_priors[:, 0]
    sobol_params = sobol_params.astype(np.float32)

    Aia = sobol_params[6]
    n_Aia = sobol_params[7]
    bg = sobol_params[8]
    n_bg = sobol_params[9]

    all_bg.append(bg)
    all_n_bg.append(n_bg)

# similarirty to fiducial

In [7]:
params = ["Om", "s8", "w0", "bg", "n_bg"]
fidu_point = dict(zip(params, parameters.get_fiducials(params)))
print(fidu_point)

fidu_Om = parameters.get_fiducials(["Om"])
fidu_s8 = parameters.get_fiducials(["s8"])
fidu_w0 = parameters.get_fiducials(["w0"])
fidu_bg = parameters.get_fiducials(["bg"])
fidu_n_bg = parameters.get_fiducials(["n_bg"])

{'Om': 0.26, 's8': 0.84, 'w0': -1.0, 'bg': 1.5, 'n_bg': 0.5}


In [12]:
np.array([1, 1, 0.5, 1, 0.5]).shape

(5,)

In [45]:
assert len(all_Om) == len(all_s8) == len(all_w0) == len(all_bg) == len(all_n_bg)

# degenerate cosmos in the sigma_8, b_g plane
fidu_dist = []
for i, (sobol, Om, s8, w0, bg, n_bg) in enumerate(zip(all_sobol, all_Om, all_s8, all_w0, all_bg, all_n_bg)):
    differences = np.squeeze(np.array([Om - fidu_Om, s8 - fidu_s8, w0 - fidu_w0, bg - fidu_bg, n_bg - fidu_n_bg]))
    differences *= np.array([1, 2, 0.25, 2, 0.5])
    differences = differences**2
    fidu_dist.append(np.mean(differences))
    
print(len(fidu_dist))

2500


In [46]:
i_cosmo = np.argmin(fidu_dist)
i_sobol = all_sobol[i_cosmo]

Om = all_Om[i_cosmo]
s8 = all_s8[i_cosmo]
w0 = all_w0[i_cosmo]
bg = all_bg[i_cosmo]
n_bg = all_n_bg[i_cosmo]

print(i_cosmo)
print(i_sobol)
print(f"Om = {Om}, s8 = {s8}, w0 = {w0}, bg = {bg}, n_bg = {n_bg}") 

2206
158664
Om = 0.181999206543, s8 = 0.83240737915, w0 = -1.036004901886, bg = 1.4890358448028564, n_bg = 0.46570587158203125


# large Aia

In [ ]:
assert len(all_Om) == len(all_s8) == len(all_w0) == len(all_bg) == len(all_n_bg)

# degenerate cosmos in the sigma_8, b_g plane
fidu_dist = []
for i, (sobol, Om, s8, w0, bg, n_bg) in enumerate(zip(all_sobol, all_Om, all_s8, all_w0, all_bg, all_n_bg)):
    differences = np.squeeze(np.array([Om - fidu_Om, s8 - fidu_s8, w0 - fidu_w0, bg - fidu_bg, n_bg - fidu_n_bg]))
    differences *= np.array([1, 2, 0.25, 2, 0.5])
    differences = differences**2
    fidu_dist.append(np.mean(differences))
    
print(len(fidu_dist))